<b>PLEASE NOTE THAT THIS NOTEBOOK WAS RUN IN GOOGLE COLAB AS SUCH CERTAIN IMPORT STATEMENTS MAY DIFFER</b>

<h1>Initial Setup</h1>

<h2>Import Packages</h2>

In [ ]:
import numpy as np
import pandas as pd
import lightgbm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import math
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import KFold, StratifiedKFold
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import os
import glob

from sklearn.cluster import DBSCAN, KMeans

<h2>Loading the Dataset</h2>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
train_path=os.path.abspath(os.path.join(os.path.dirname("__file__"), '..', '/content/drive/My Drive/datasets/train.csv'))
training_df = pd.read_csv(train_path, index_col="tripid")

<h1>Feature Addition</h1>

In [ ]:
def dist_from_coordinates(lat1, lon1, lat2, lon2):
  R = 6371  # Earth radius in km

  #conversion to radians
  d_lat = np.radians(lat2-lat1)
  d_lon = np.radians(lon2-lon1)

  r_lat1 = np.radians(lat1)
  r_lat2 = np.radians(lat2)

  #haversine formula
  a = np.sin(d_lat/2.) **2 + np.cos(r_lat1) * np.cos(r_lat2) * np.sin(d_lon/2.)**2

  haversine = 2 * R * np.arcsin(np.sqrt(a))

  return haversine

In [ ]:
training_df['pickup_time'] = pd.to_datetime(training_df['pickup_time'], format="%m/%d/%Y %H:%M")
training_df['drop_time'] = pd.to_datetime(training_df['drop_time'], format="%m/%d/%Y %H:%M")

In [ ]:
training_df = training_df.assign(timeOfDay=pd.cut(training_df.pickup_time.dt.hour,[-1, 8, 20, 24],labels=['dawn','day', 'night']))

In [ ]:
training_df.loc[training_df['timeOfDay'] == 'day', 'isNormalCharge'] = 1
training_df.loc[training_df['timeOfDay'] != 'day', 'isNormalCharge'] = 0

In [ ]:
durations = []
for index,row in training_df.iterrows():
  provided_duration = row['duration'] #first row of location.lat column here
  if math.isnan(provided_duration) or provided_duration <= 0 :
    time_dif = (row['drop_time'] - row['pickup_time']).seconds
    if(time_dif == 0):
        time_dif = np.nan
    durations.append(time_dif)
  else :  
    durations.append(provided_duration)

training_df.insert(4,"time_dif",durations)

In [ ]:
new_column = []                    #empty column for distance
for index,row in training_df.iterrows():
  lat1 = row['pick_lat'] #first row of location.lat column here
  lon1 = row['pick_lon'] #first row of location.long column here
  lat2 = row['drop_lat'] #second row of location.lat column here
  lon2 = row['drop_lon'] #second row of location.long column here
  value = dist_from_coordinates(lat1, lon1, lat2, lon2)  #get the distance
  new_column.append(value)   #append the empty list with distance values

training_df.insert(4,"distance",new_column)

In [ ]:
training_df['time_driven'] = training_df['duration']  - training_df['meter_waiting']

In [ ]:
chargeperhours = []
for index,row in training_df.iterrows():
    if(row['meter_waiting'] == 0):
        chargeperhour = np.nan
    else:
        chargeperhour = (row['meter_waiting_fare'] / row['meter_waiting'] * 3600)
    chargeperhours.append(chargeperhour)

training_df.insert(4,'charge_per_hour',chargeperhours)


In [ ]:
training_df['driving_fare'] = training_df['fare']  - training_df['meter_waiting_fare'] - training_df['additional_fare']

In [ ]:
avgspeeds = []
for index,row in training_df.iterrows():
    if(row['time_driven'] == 0):
        avgspeed = np.nan
    else:
        avgspeed = (row['distance'] / row['time_driven'] * 3600)
    avgspeeds.append(avgspeed)

training_df.insert(4,"avg_speed",avgspeeds)



In [ ]:
costsperkm = []
for index,row in training_df.iterrows():
    if row['distance'] == 0:
        costperkm = np.nan
            
    else:
        costperkm = (row['driving_fare'] / row['distance'])
    costsperkm.append(costperkm)

training_df.insert(4,"cost_per_km",costsperkm)

In [ ]:
training_df = training_df.replace({'label': {'incorrect': 0, 'correct' : 1}})

<h2>Geolocation clustering</h2>

In [ ]:
test_path =os.path.abspath(os.path.join(os.path.dirname("__file__"), '..', '/content/drive/My Drive/datasets/test.csv'))
test_set = pd.read_csv(test_path, index_col="tripid")

In [ ]:
pickup_locations = pd.DataFrame()
drop_locations = pd.DataFrame()
test_pickup_locations = pd.DataFrame()
test_drop_locations = pd.DataFrame()

In [ ]:
pickup_locations[['lat','lon']] = training_df[['pick_lat','pick_lon']]
drop_locations[['lat','lon']] = training_df[['drop_lat','drop_lon']]

In [ ]:
test_pickup_locations[['lat','lon']] = test_set[['pick_lat','pick_lon']]
test_drop_locations[['lat','lon']] = test_set[['drop_lat','drop_lon']]

In [ ]:
pickup_locations = pickup_locations.reset_index()
drop_locations = drop_locations.reset_index()
test_pickup_locations = test_pickup_locations.reset_index()
test_drop_locations = test_drop_locations.reset_index()

In [ ]:
df = pd.concat([pickup_locations,drop_locations,test_pickup_locations,test_drop_locations],ignore_index=True )

In [ ]:
db_pick = DBSCAN(eps=1/6371., min_samples=5, algorithm='ball_tree', metric='haversine').fit(np.radians(df[['lat','lon']]))

In [ ]:
db_pick.labels_

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
db_pick.labels_[0:17176]

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
training_df['pickup_cluster_label'] = db_pick.labels_[0:17176]

In [ ]:
training_df['pickup_cluster_label'].value_counts()

 0     14409
 1      1870
 4       328
 2       277
-1        75
 6        73
 3        64
 5        26
 12       10
 15        6
 17        6
 16        4
 13        4
 11        4
 9         4
 18        3
 7         3
 10        2
 8         2
 20        1
 19        1
 22        1
 14        1
 28        1
Name: pickup_cluster_label, dtype: int64

In [ ]:
training_df['drop_cluster_label'] = db_pick.labels_[17176:17176*2]

<h2>Final Feature Selection</h2>

In [ ]:
training_df.columns

Index(['additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare',
       'cost_per_km', 'avg_speed', 'charge_per_hour', 'distance', 'time_dif',
       'meter_waiting_till_pickup', 'pickup_time', 'drop_time', 'pick_lat',
       'pick_lon', 'drop_lat', 'drop_lon', 'fare', 'label', 'timeOfDay',
       'isNormalCharge', 'time_driven', 'driving_fare', 'pickup_cluster_label',
       'drop_cluster_label'],
      dtype='object')

In [ ]:
training_columns = ['meter_waiting','meter_waiting_fare','fare','additional_fare', 'distance','cost_per_km', 'avg_speed',  'time_dif','time_driven', 'charge_per_hour', 'driving_fare', 'isNormalCharge','pick_lat','pick_lon','drop_lat','drop_lon','drop_cluster_label','pickup_cluster_label']

In [ ]:
target_column = ['label']

In [ ]:
training_df = training_df.drop(190167541)

In [ ]:
x = training_df[training_columns].values
y = training_df[target_column].values

In [ ]:
x, x_test, y, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
training_df['label'].value_counts()

1    15494
0     1681
Name: label, dtype: int64

<h1>Model Training</h1>

<h2>Eval Metric Definition</h2>

In [ ]:
def focal_loss_lgb_eval_error(y_true, y_pred, alpha=.25, gamma=2.):
    a,g = alpha, gamma
    p = 1/(1+np.exp(-y_pred))
    loss = -( a*y_true + (1-a)*(1-y_true) ) * (( 1 - ( y_true*p + (1-y_true)*(1-p)) )**g) * ( y_true*np.log(p)+(1-y_true)*np.log(1-p) )
    return 'focal_loss', np.mean(loss), False

In [ ]:
from sklearn.metrics import f1_score
def evaluate_macroF1_lgb(y_true, y_pred):  
    y_hat = np.where(y_pred < 0.5, 0, 1) 
    f1 = f1_score(y_true, y_hat, average='macro')
    return ('macroF1', f1, True) 

<h2>Training the Model</h2>

In [ ]:
gkf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
model = XGBClassifier( scale_pos_weight=1, learning_rate=0.6, eval_metric='logloss')

In [ ]:
for training_index, testing_index in gkf.split(X=x, y=y):
    x_train_fold, y_train_fold = x[training_index], y[training_index]
    x_test_fold, y_test_fold = x[testing_index], y[testing_index]
    model.fit(x_train_fold, y_train_fold, eval_set=[(x_test_fold,y_test_fold)])

[0]	validation_0-logloss:0.374034
[1]	validation_0-logloss:0.273479
[2]	validation_0-logloss:0.223017
[3]	validation_0-logloss:0.196113
[4]	validation_0-logloss:0.176907
[5]	validation_0-logloss:0.169543
[6]	validation_0-logloss:0.164202
[7]	validation_0-logloss:0.158004
[8]	validation_0-logloss:0.157001


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[9]	validation_0-logloss:0.155688
[10]	validation_0-logloss:0.152632
[11]	validation_0-logloss:0.150826
[12]	validation_0-logloss:0.151042
[13]	validation_0-logloss:0.150815
[14]	validation_0-logloss:0.150411
[15]	validation_0-logloss:0.148578
[16]	validation_0-logloss:0.148152
[17]	validation_0-logloss:0.147525
[18]	validation_0-logloss:0.145773
[19]	validation_0-logloss:0.146239
[20]	validation_0-logloss:0.145611
[21]	validation_0-logloss:0.144963
[22]	validation_0-logloss:0.145574
[23]	validation_0-logloss:0.145854
[24]	validation_0-logloss:0.144814
[25]	validation_0-logloss:0.144705
[26]	validation_0-logloss:0.144883
[27]	validation_0-logloss:0.144718
[28]	validation_0-logloss:0.145118
[29]	validation_0-logloss:0.145861
[30]	validation_0-logloss:0.14569
[31]	validation_0-logloss:0.14587
[32]	validation_0-logloss:0.145811
[33]	validation_0-logloss:0.145274
[34]	validation_0-logloss:0.144837
[35]	validation_0-logloss:0.145083
[36]	validation_0-logloss:0.145775
[37]	validation_0-loglo

<h2>Model Evaluation</h2>

In [ ]:
predicted_y = model.predict(x_test)
train_pred_y = model.predict(x)

In [ ]:
print(); print(metrics.classification_report(y,train_pred_y))
print(); print(metrics.confusion_matrix(y, train_pred_y))

print(); print(metrics.classification_report(y_test, predicted_y))
print(); print(metrics.confusion_matrix(y_test, predicted_y))


              precision    recall  f1-score   support

           0       0.95      0.79      0.86      1345
           1       0.98      1.00      0.99     12395

    accuracy                           0.98     13740
   macro avg       0.96      0.89      0.93     13740
weighted avg       0.98      0.98      0.97     13740


[[ 1065   280]
 [   54 12341]]

              precision    recall  f1-score   support

           0       0.87      0.67      0.76       336
           1       0.97      0.99      0.98      3099

    accuracy                           0.96      3435
   macro avg       0.92      0.83      0.87      3435
weighted avg       0.96      0.96      0.96      3435


[[ 226  110]
 [  34 3065]]


<h1>Testing</h1>

<h2>Loading the Test Dataset</h2>

In [ ]:
test_path =os.path.abspath(os.path.join(os.path.dirname("__file__"), '..', '/content/drive/My Drive/datasets/test.csv'))
test_set = pd.read_csv(test_path, index_col="tripid")

<h2>Feature Addition on testing dataset</h2>

In [ ]:
test_set['pickup_time'] = pd.to_datetime(test_set['pickup_time'], format="%m/%d/%Y %H:%M")
test_set['drop_time'] = pd.to_datetime(test_set['drop_time'], format="%m/%d/%Y %H:%M")

In [ ]:
test_set = test_set.assign(timeOfDay=pd.cut(test_set.pickup_time.dt.hour,[-1, 8, 20, 24],labels=['dawn','day', 'night']))

In [ ]:
test_set.loc[test_set['timeOfDay'] == 'day', 'isNormalCharge'] = 1
test_set.loc[test_set['timeOfDay'] != 'day', 'isNormalCharge'] = 0

In [ ]:
durations = []
for index,row in test_set.iterrows():
  provided_duration = row['duration'] #first row of location.lat column here
  if math.isnan(provided_duration) or provided_duration <= 0 :
    time_dif = (row['drop_time'] - row['pickup_time']).seconds
    if(time_dif == 0):
        time_dif = np.nan
    durations.append(time_dif)
  else :  
    durations.append(provided_duration)

test_set.insert(4,"time_dif",durations)

In [ ]:
new_column = []                    #empty column for distance
for index,row in test_set.iterrows():
  lat1 = row['pick_lat'] #first row of location.lat column here
  lon1 = row['pick_lon'] #first row of location.long column here
  lat2 = row['drop_lat'] #second row of location.lat column here
  lon2 = row['drop_lon'] #second row of location.long column here
  value = dist_from_coordinates(lat1, lon1, lat2, lon2)  #get the distance
  new_column.append(value)   #append the empty list with distance values

test_set.insert(4,"distance",new_column)

In [ ]:
test_set['time_driven'] = test_set['duration']  - test_set['meter_waiting']

In [ ]:
chargeperhours = []
for index,row in test_set.iterrows():
    if(row['meter_waiting'] == 0):
        chargeperhour = np.nan
    else:
        chargeperhour = (row['meter_waiting_fare'] / row['meter_waiting'] * 3600)
    chargeperhours.append(chargeperhour)

test_set.insert(4,'charge_per_hour',chargeperhours)


In [ ]:
test_set['driving_fare'] = test_set['fare']  - test_set['meter_waiting_fare'] - test_set['additional_fare']

In [ ]:
avgspeeds = []
for index,row in test_set.iterrows():
    if(row['time_driven'] == 0):
        avgspeed = np.nan    
    else:
        avgspeed = (row['distance'] / row['time_driven'] * 3600)
    avgspeeds.append(avgspeed)

test_set.insert(4,"avg_speed",avgspeeds)



In [ ]:
costsperkm = []
for index,row in test_set.iterrows():
    if row['distance'] == 0:
        costperkm = np.nan
            
    else:
        costperkm = (row['driving_fare'] / row['distance'])
    costsperkm.append(costperkm)

test_set.insert(4,"cost_per_km",costsperkm)

<h2>Adding cluster labels to test dataset</h2>

In [ ]:
test_set['pickup_cluster_label'] = db_pick.labels_[17176*2:17176*2 + 8576 ]
test_set['drop_cluster_label'] = db_pick.labels_[17176*2 + 8576: 2*(17176+8576) ]

In [ ]:
test_features = test_set[training_columns]

<h2>Prediction using trained model</h2>

In [ ]:
predicted_labels = model.predict(test_features.values)

In [ ]:
predicted_labels_df = pd.DataFrame(predicted_labels )

In [ ]:
sub_path =os.path.abspath(os.path.join(os.path.dirname("__file__"), '..', '/content/drive/My Drive/datasets/sample_submission.csv'))
submission_set = pd.read_csv(sub_path)

In [ ]:
submission_set['prediction']= predicted_labels_df.values[:,0]

<h2>Submission file generation</h2>

In [ ]:
theNotebook = "dbscan_xg"

In [ ]:
filename = '/content/drive/My Drive/datasets/'+theNotebook+'/'+theNotebook+'_{%i}.csv'
dirname = '/content/drive/My Drive/datasets/'+theNotebook
fileversion = 1

if not os.path.exists(dirname):
    os.makedirs(dirname)
while glob.glob(filename.replace('{%i}',str(fileversion))) :
    fileversion+=1
submission_set.to_csv(filename.replace('{%i}',str(fileversion)), index=False)
print("Completed!")

Completed!


In [ ]:
submission_set['prediction'].value_counts()

1    8113
0     463
Name: prediction, dtype: int64